In [215]:
import requests
import re
import pandas as pd
import xlrd
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import time
import FinanceDataReader as fdr
from bs4 import BeautifulSoup
import os
import numpy as np

In [2]:
crtfc_key = open('crtfc_key.txt',mode='r').readline()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'

# Corpcode

## Corpcode.xml 가져오기

In [ ]:
url_corp = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key='+crtfc_key
# requests.get(url_corp).content.decode("UTF-8")
with urlopen(url_corp) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('Raw/Corpcode/corp_num')

## Corpcode.xml에서 기업코드 가져오기

In [3]:
tree = ET.parse('Raw/Corpcode/corp_num/CORPCODE.xml')
root = tree.getroot()

corp_df = pd.DataFrame(columns=['corp_code','corp_name','stock_code','modify_date'])
for company in root.iter('list'):
    stock_code = company.findtext('stock_code')
    stock_code = stock_code.strip()
    if stock_code:
        company_dict = {
            'corp_code':company.findtext('corp_code'),
            'corp_name':company.findtext('corp_name'),
            'stock_code':company.findtext('stock_code'),
            'modify_date':company.findtext('modify_date')
        }
        corp_df = corp_df.append(company_dict, ignore_index=True)
# corp_df

In [ ]:
# corp_df[corp_df['corp_name']=='BYC']
a = corp_df[corp_df['stock_code']=='168490'].iloc[0,0]
corp_df[corp_df['corp_code']==a]

# 필요 함수

## 함수생성을 위한 TEST

In [ ]:
corp_code = stock_code_reset('005930')
bgn_de = "19990101"
url_api = "https://opendart.fss.or.kr/api/list.xml?crtfc_key=" + crtfc_key + "&corp_code=" + corp_code + "&bgn_de=" + bgn_de +"&last_reprt_at=Y" + "&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003" + "&page_no=10&page_count=100"
dart_page = requests.get(url_api).content.decode("UTF-8")
rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', dart_page)
period_list = re.findall(r'<report_nm>(.*?)</report_nm>', dart_page)
year_list = list(map(year_reset, period_list))


In [ ]:
period_list[80][-8:-1] +'_'+ period_list[35][-15:-10]

In [ ]:
rcp_no = '20050816000742'

In [ ]:
url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp)
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
resp = requests.get(url,headers={'user-agent':user_agent})
webpage = resp.content.decode("UTF-8")
webpage_clean = ['\n','\t',' ','(',')',"'",'"']
for clean in webpage_clean:
    webpage = webpage.replace(clean,'')
# try:
# webpage = re.findall(r"연결재무제표(.*?).xsd",webpage)[0]
# webpage = re.findall("부속명세서(.*?).xsd",webpage)[0]
# except:
# webpage = re.findall(r"재무제표(.*?).xsd",webpage)[0]
# webpage = re.findall(r"viewDoc(.*)",webpage)[0]
# dcm_eleid_no = webpage.split(',')[1:3]

In [ ]:
webpage

In [ ]:
eleid

In [ ]:
soup = BeautifulSoup(resp.content, 'html.parser')
soup

In [ ]:
# making_rcp_df(stock_code_reset('036670'))
making_rcp_df(stock_code_reset('005930'))

## 실행 함수

In [4]:
def making_rcp_df(corp_code):
    bgn_de = "19990101"
    end_de = "20210410"
#     bgn_de = "20200101"
    url_api = "https://opendart.fss.or.kr/api/list.xml?crtfc_key=" + crtfc_key + "&corp_code=" + corp_code + "&bgn_de=" + bgn_de +"&end_de="+end_de+"&last_reprt_at=Y" + "&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003" + "&page_no=10&page_count=100"
    dart_page = requests.get(url_api).content.decode("UTF-8")
    rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', dart_page)
    period_list = re.findall(r'<report_nm>(.*?)</report_nm>', dart_page)
    year_list = list(map(year_reset, period_list))
    
#     dcm_no_list = list(map(making_dcm, rcp_no_list))
    dcm_eleid_no = list(map(making_dcm_eleid,rcp_no_list))
    dcm_no_list = pd.DataFrame(dcm_eleid_no)[0].to_list()
    eleid_list = pd.DataFrame(dcm_eleid_no)[1].to_list()
    
    stock_code = corp_code_reset(corp_code)
    
    rcp_no_list.insert(0,stock_code)
    rcp_no_list.insert(1,'rcp_no')
    year_list.insert(0,'stock_code')
    year_list.insert(1,'description')
    dcm_no_list.insert(0,stock_code)
    dcm_no_list.insert(1,'dcm_no')
    eleid_list.insert(0,stock_code)
    eleid_list.insert(1,'eleid')
    
#     rcp_df = pd.DataFrame(data=[rcp_no_list,dcm_no_list],columns=year_list)
    rcp_df = pd.DataFrame(data=[rcp_no_list,dcm_no_list,eleid_list],columns=year_list)
    rcp_df = rcp_df.loc[:, ~rcp_df.columns.duplicated(
        keep='first')]  #한 해에 여러개의 보고서가 나오면 최근거만 가져옴
    return rcp_df

# def making_dcm (rcp_no):
#     url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no)
#     user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
#     resp = requests.get(url,headers={'user-agent':user_agent})
#     webpage = resp.content.decode("UTF-8")
#     dcm_no = re.findall(r"{}', '(.*?)',".format(rcp_no),webpage)[0]
#     time.sleep(0.5)
#     return dcm_no

def making_dcm_eleid (rcp_no):
#     print(rcp_no)
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    resp = requests.get(url,headers={'user-agent':user_agent})
    webpage = resp.content.decode("UTF-8")
    webpage_clean = ['\n','\t',' ','(',')',"'",'"']
    for clean in webpage_clean:
        webpage = webpage.replace(clean,'')
    search_words = ['연결재무제표','재무제표','부속명세서']
    for search_word in search_words:
        try :
            webpage = re.findall(search_word+"(.*?).dart",webpage)[0]
            break
        except:
            continue
    webpage = re.findall(r"viewDoc(.*)",webpage)[0]
    dcm_eleid_no = webpage.split(',')[1:3]
    time.sleep(0.5)
    return dcm_eleid_no

def corp_code_reset(corp_code):  #에러가 발생할 경우 회사 코드 불러오기 실핸한다.
    stock_code = corp_df[corp_df['corp_code'] == corp_code].iloc[0, 2]
    return stock_code

def stock_code_reset(stock_code):
    corp_code = corp_df[corp_df['stock_code'] == stock_code].iloc[0, 0]
    return corp_code

def year_reset(title):
    try:
        year = period_list[35][-8:-1] +'_'+ period_list[35][-15:-10]
#         year = [i for i in re.findall('\(([^)]+)', title) if i][0]
    except:
        year = '0'
    return year


# rcp, dcm 가져오기

## stock_code 가져오기

In [ ]:
# stock_df = fdr.StockListing('KOSPI')
stock_code_list = ['005930','005380','017670']

## rcp, dcm, eleid 만들기

In [ ]:
# start = time.time()
rcp_df = pd.DataFrame()
indexerror_list=[]
valueerror_list=[]
for i in stock_code_list:
    try :
        rcp_df_temp = making_rcp_df(stock_code_reset(i))
        rcp_df = pd.concat([rcp_df,rcp_df_temp],axis=0)
    except IndexError:
        indexerror_list.append(i)
    except ValueError:
        valueerror_list.append(i)
    except Exception as e:
        print (i,e)
rcp_df.reset_index(drop=True,inplace=True)
# print(time.time()-start)
rcp_df.to_csv('Raw/rcp_dcm/rcp_df.csv', index=True, encoding='utf-8-sig')
# IndexError : single positional indexer is out-of-bounds : 우선주 등 재무제표 없는 경우
# list index out of range : 보고서 제목에 년도가 안써있는 경우 -> 0으로 쓰는것으로 수정
# ValueError : Length mismatch: Expected axis has 0 elements, new values have 1 elements : 리츠 등 재무제표 없는 경우

In [ ]:
rcp_df

## rcp, dcm 불러오기

In [5]:
def six_degree (stock_code):
    stock_code_return = '%06d'%(stock_code)
    return stock_code_return

In [6]:
rcp_df = pd.read_csv('Raw/rcp_dcm/rcp_df.csv')
# rcp_df = rcp_df.set_index(rcp_df.columns[0])
# rcp_df.index.name = None
rcp_df = rcp_df.drop(columns=rcp_df.columns[0])
rcp_df.reset_index(drop=True,inplace=True)
# column 지정
# rcp_df.columns = [c.columns.map(lambda x : x[:2]), c.iloc[0], c.iloc[1]]
# c = c.iloc[2:]
# 종목명 6자리로 변경
# for i in range(len(rcp_df.index)):
#     rcp_df.rename(index={rcp_df.index[i]:'%06d'%(rcp_df.index[i])}, inplace = True)
rcp_df['stock_code']=list(map(six_degree, rcp_df['stock_code']))

# rcp_df

# 재무제표 다운로드

## Error_case
- 연결재무제표가 제출예정인 경우, rcp = 20070330001989 (005930, 2006.12), 20060331001832 (005930, 2005.12),('005930', '2004.12'),(005380, 2006.12),(005380, 2005.12),(005380, 2004.12),(005380, 2002.12),(005380, 2001.12),(005380, 2000.12),(005380, 1998.12)
~> 연결 감사보고서로 검색
- 연결재무제표에 '재무상태표'등 항목명이 안써있는 경우 : (005930, 2005.09), (005930, 2005.06), (005930, 2004.09),(005930, 2004.06)
~> 찾는 방법 추가
- 2000년대 초반에는 반기, 분기 보고서의 연결재무제표에서는 분기자료가 아닌 연간 자료가 입력되어 있다. (현대자동차로 보니 2009년 부터 분기 자료로 들어가 있는듯)
~> 상단의 기간 테이블을 같이 추출하기 or 일단 최근 10년의 데이터만으로 시작해보기.(오류항목 더 적음, 다운로드만으로 가능할듯)

In [ ]:
stock_code, year

In [ ]:
# error_df.to_csv('error_df.csv')
# error_df=pd.read_csv('error_df.csv',encoding='utf-8',index_col=0).astype(int)
error_df

In [ ]:
error_df = pd.DataFrame(index = ['stock_code','year','rcp','dcm','eleid'])

for no_of_stock in range(int(len(rcp_df.index)/3)):
    rcp_list_temp = rcp_df.iloc[3*no_of_stock,2:].values
    dcm_list_temp = rcp_df.iloc[3*no_of_stock+1,2:].values
    eleid_list_temp = rcp_df.iloc[3*no_of_stock+2,2:].values
    year_list_temp = rcp_df.columns[2:]
    stock_code_temp = rcp_df.iloc[3*no_of_stock,0]
    for year, rcp, dcm, eleid in zip(year_list_temp,rcp_list_temp,dcm_list_temp,eleid_list_temp):
        try:
            download_ftable_html(stock_code_temp,year,rcp,dcm,eleid)
        except:
            no_error=len(error_df.columns)
            error_df[no_error] = [stock_code_temp,year,rcp,dcm,eleid]

In [7]:
# 2007.09 부터 가능
def download_ftable_excel(stock_code,year,rcp,dcm):
    url_download = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko'.format(rcp,dcm)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    resp_download = requests.get(url_download, headers = {'user-agent' : user_agent}, verify=False)
    table_download = BytesIO(resp_download.content)
    sheet_names = pd.ExcelFile(table_download).sheet_names
    sheets_group = [['연결 재무상태표','재무상태표','대차대조표'],['연결 현금흐름표','현금흐름표','현금흐름표'],['연결 포괄손익계산서','포괄손익계산서','손익계산서']]
    for sheets in sheets_group:
        i=0
        while True:
            if sheets[i] not in sheet_names:
                i=i+1
                if i > len(sheets):
                    print("재무상태표를 찾을 수 없습니다")
                    break
            else: 
                data_download = pd.read_excel(table_download, sheet_name = sheets[i])
                try:
                    data_download.to_csv("Raw/Dart_Crawling/"+stock_code+"_"+year+"_"+sheets[1]+".csv", encoding='euc-kr')
                except UnicodeEncodeError:
                    # 최근것은 euc-kr로 encoding되어 있으나, 2010년이전에는 utf-16으로 encoding된것으로 보임.
                    # 다만, utf-16에서 to_csv할 때 sep=','이 적용되지 않아 다운로드된 파일의 글씨는 깨지지만 utf-8로 사용.
                    data_download.to_csv("Raw/Dart_Crawling/"+stock_code+"_"+year+"_"+sheets[1]+".csv", encoding='utf-8')
                break

In [8]:
def download_ftable_html(stock_code,year,rcp,dcm,eleid):
    url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo={}&dcmNo={}&eleId={}&offset=687340&length=115750&dtd=dart3.xsd'.format(int(rcp),int(dcm),int(eleid))
    table = pd.read_html(url)
    sheets_group = [['연결재무상태표','재무상태표','대차대조표'],['연결현금흐름표','현금흐름표','현금흐름표'],['연결포괄손익계산서','포괄손익계산서','손익계산서']]
    for sheets in sheets_group:
        for sheet in sheets:
            find=0
            for i in range(len(table)):
                if sheet in str(table[i].iloc[0,0]).replace(' ',''):
                    unit = table[i].iloc[-1,0]
                    table_temp = table[i+1]
                    table_temp['unit'] = unit
                    table_temp.to_csv("Raw/Dart_Crwaling_Table/"+str(stock_code)+"_"+str(year)+"_"+sheets[1]+".csv", encoding='utf-8-sig')
                    find=1
                    break
            if find == 1:
                break


In [ ]:
stock_code='005380'
year='1998.12'
rcp,dcm,eleid = rcp_df[rcp_df['stock_code']==stock_code][year].values

In [ ]:
stock_code, year,rcp, dcm, eleid = error_df.iloc[:,0].values
# error_df.iloc[:,0].values

In [ ]:
download_ftable_html(stock_code, year,rcp, dcm, eleid)

In [ ]:
import webbrowser
url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp)
webbrowser.open(url)

In [ ]:
url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo={}&dcmNo={}&eleId={}&offset=687340&length=115750&dtd=dart3.xsd'.format(int(rcp),int(dcm),int(eleid))
table = pd.read_html(url)

In [ ]:
# for i in range(len(table)):
#     print(table[i].iloc[0,0])
# str(table[1].iloc[0,0]).replace(' ','')
table[1]

# Download CSV 활용

1. 설명서
- 단위를 불러오는 방법, 본격적인 테이블이 시작하는 부분을 가져오는 방법 등을 고민
- 필요 항목명 정리
- 항목을 가져오는 다양한 아이디어 고민 (다음 년도에 같은 값을 가진 항목을 불러오기 등)

In [17]:
data_path = './Raw/Dart_Crawling/'
files_list = os.listdir(data_path)

In [261]:
file_temp

'005380_2007.09_재무상태표.csv'

In [270]:
file_temp = files_list[0]
file = data_path + file_temp
reading_csv(file)

utf-8


,제40기 분기말,제39기말,제38기말
유동자산,84289.71,80430.05,90033.65
당좌자산,69397.93,68234.19,79952.79
현금및현금성자산,10091.22,13826.78,18032.82
단기금융상품,33168.43,27553.81,38607.35
단기투자자산,1560.08,3102.95,3422.92
...,...,...,...
이익준비금,2509.70,2233.70,1888.70
기타적립금,95616.47,83216.47,60516.47
미처분이익잉여금미처리결손금당기순이익제기분기백만원제기백만원제기백만원,13707.86,15454.48,26840.20
자본총계,177651.33,160508.28,150529.64


In [248]:
for file_temp in files_list:
    print(file_temp)
    file = data_path + file_temp
    reading_csv(file)

005380_2007.09_재무상태표.csv
005380_2007.09_포괄손익계산서.csv
005380_2007.09_현금흐름표.csv
005380_2007.12_재무상태표.csv
005380_2007.12_포괄손익계산서.csv
005380_2007.12_현금흐름표.csv
005380_2008.03_재무상태표.csv
005380_2008.03_포괄손익계산서.csv
005380_2008.03_현금흐름표.csv
005380_2008.06_재무상태표.csv
005380_2008.06_포괄손익계산서.csv
005380_2008.06_현금흐름표.csv
005380_2008.09_재무상태표.csv
005380_2008.09_포괄손익계산서.csv
005380_2008.09_현금흐름표.csv
005380_2008.12_재무상태표.csv
005380_2008.12_포괄손익계산서.csv
005380_2008.12_현금흐름표.csv
005380_2009.03_재무상태표.csv
005380_2009.03_포괄손익계산서.csv
005380_2009.03_현금흐름표.csv
005380_2009.06_재무상태표.csv
005380_2009.06_포괄손익계산서.csv
005380_2009.06_현금흐름표.csv
005380_2009.09_재무상태표.csv
005380_2009.09_포괄손익계산서.csv
005380_2009.09_현금흐름표.csv
005380_2009.12_재무상태표.csv
005380_2009.12_포괄손익계산서.csv
005380_2009.12_현금흐름표.csv
005380_2010.03_재무상태표.csv
005380_2010.03_포괄손익계산서.csv
005380_2010.03_현금흐름표.csv
005380_2010.06_재무상태표.csv
005380_2010.06_포괄손익계산서.csv
005380_2010.06_현금흐름표.csv
005380_2010.09_재무상태표.csv
005380_2010.09_포괄손익계산서.csv
005380_2010.09_현금흐름표.csv

017670_2009.03_재무상태표.csv
017670_2009.03_포괄손익계산서.csv
017670_2009.03_현금흐름표.csv
017670_2009.06_재무상태표.csv
017670_2009.06_포괄손익계산서.csv
017670_2009.06_현금흐름표.csv
017670_2009.09_재무상태표.csv
017670_2009.09_포괄손익계산서.csv
017670_2009.09_현금흐름표.csv
017670_2009.12_재무상태표.csv
017670_2009.12_포괄손익계산서.csv
017670_2009.12_현금흐름표.csv
017670_2010.03_재무상태표.csv
017670_2010.03_포괄손익계산서.csv
017670_2010.03_현금흐름표.csv
017670_2010.06_재무상태표.csv
017670_2010.06_포괄손익계산서.csv
017670_2010.06_현금흐름표.csv
017670_2010.09_재무상태표.csv
017670_2010.09_포괄손익계산서.csv
017670_2010.09_현금흐름표.csv
017670_2010.12_재무상태표.csv
017670_2010.12_포괄손익계산서.csv
017670_2010.12_현금흐름표.csv
017670_2011.03_재무상태표.csv
017670_2011.03_포괄손익계산서.csv
017670_2011.03_현금흐름표.csv
017670_2011.06_재무상태표.csv
017670_2011.06_포괄손익계산서.csv
017670_2011.06_현금흐름표.csv
017670_2011.09_재무상태표.csv
017670_2011.09_포괄손익계산서.csv
017670_2011.09_현금흐름표.csv
017670_2011.12_재무상태표.csv
017670_2011.12_포괄손익계산서.csv
017670_2011.12_현금흐름표.csv
017670_2012.03_재무상태표.csv
017670_2012.03_포괄손익계산서.csv
017670_2012.03_현금흐름표.csv

In [257]:
csv_temp = reading_csv(file)
index = csv_temp.index
index = list(map(lambda x : (''.join(re.compile('[가-힣]+').findall(str(x)))).replace('주석','').rstrip('주'), index))
csv_temp.index = index
csv_temp

,제40기 3분기,NaN,제39기 3분기,NaN,제39기 연간,제38기 연간
매출액,70419.50,217529.61,58869.35,197512.55,273353.68,273837.37
국내매출액,30899.27,94611.55,27104.13,85108.82,118396.88,110192.43
수출매출액,39520.23,122918.06,31765.22,112403.73,154956.80,163644.94
매출원가,57764.90,177128.21,48890.39,161081.94,224085.72,221857.43
기초재고액,7788.02,6127.35,7306.40,4740.49,4740.49,6984.72
...,...,...,...,...,...,...
파생상품평가손실,762.05,551.94,277.29,3811.72,4448.20,1805.52
잡손실,227.06,799.09,1393.85,1828.85,1586.68,3553.60
법인세비용차감전순이익손실,4908.29,17103.66,3287.82,13393.24,18858.83,27390.58
법인세비용,653.51,3659.65,350.03,3000.11,3598.20,3903.37


In [243]:
csv_temp = pd.read_csv(file, encoding ='utf-8', index_col = 0)
for i in range(len(csv_temp.index)):
    if '단위' in str(csv_temp.iat[i,0]):
        slice_point = i
        break
unit_korean = csv_temp.iat[slice_point,0]
if '십억원' in unit_korean:
    unit = 10**9 / 10**8
elif'백만원' in unit_korean: 
    unit = 10**6 / 10**8
elif '천원' in unit_korean:
    unit = 10**3 / 10**8
else:
    print(file + '_check unit_korean')
csv_temp_filtered = csv_temp.iloc[slice_point+1:,:]
csv_temp_filtered = csv_temp_filtered.dropna(how='all')
csv_temp_filtered.columns = csv_temp_filtered.iloc[0,:]
csv_temp_filtered.index = csv_temp_filtered.iloc[:,0]
csv_temp_filtered = csv_temp_filtered.iloc[1:,1:]
csv_temp_filtered.columns.name=''


In [269]:
def reading_csv (file):
    try:
        csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0) # index_col : index로 삼을 열 지정
        print('cp949')
    except UnicodeDecodeError:
        csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
        print('utf-8')
    # 단위가 적혀 있는 cell찾기
    for i in range(len(csv_temp.index)):
        if '단위' in str(csv_temp.iat[i,0]):
            slice_point = i
            break
    # 원 단위로 하면 보기가 불편하여 억 단위로 반영
    unit_korean = csv_temp.iat[slice_point,0]
    if '십억원' in unit_korean:
        unit = 10**9 / 10**8
    elif'백만원' in unit_korean: 
        unit = 10**6 / 10**8
    elif '천원' in unit_korean:
        unit = 10**3 / 10**8
    else:
        print(file + '_check unit_korean')
    # 재무제표 자료만을 수정
    csv_temp_filtered = csv_temp.iloc[slice_point+1:,:]
    csv_temp_filtered = csv_temp_filtered.dropna(how='all')
    csv_temp_filtered.columns = csv_temp_filtered.iloc[0,:]
    csv_temp_filtered.index = csv_temp_filtered.iloc[:,0]
    csv_temp_filtered = csv_temp_filtered.iloc[1:,1:]
    csv_temp_filtered.columns.name=''
    csv_temp_filtered = csv_temp_filtered.apply(pd.to_numeric,errors = 'coerce') #text가 있어서 float를 변환이 안될 경우에 NaN으로 변환
#     csv_temp_filtered = csv_temp_filtered.replace('\xa0',np.nan) # \xa0는 줄바꿈방지 공백(Non-breaking Space,?)로 결측치로 수정
#     csv_temp_filtered = csv_temp_filtered.replace(' ',np.nan) # 값이 없는 경우 ' '으로 되어 있어 결측치로 수정
    csv_temp_filtered = csv_temp_filtered.dropna(how='all') # 결측치를 삭제
#     csv_temp_filtered = csv_temp_filtered.astype(float) # 값 형식이 str으로 되어 있어 float로 수정
    csv_temp_filtered = csv_temp_filtered * unit
    # index를 clearing
    index = list(map(lambda x : (''.join(re.compile('[가-힣]+').findall(str(x)))).replace('주석','').rstrip('주'), csv_temp_filtered.index))
    csv_temp_filtered.index = index
    # download
    file = file.replace('Dart_Crawling','Dart_Crawling (Filtered)')
    csv_temp_filtered.to_csv(file,encoding ='cp949')
    return csv_temp_filtered

In [26]:
file_temp = files_list[0]
pd.read_csv(data_path + file_temp, encoding ='utf-8').head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,0,대차대조표,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN
2,2,제40기 3분기 2007년 09월 30일 현재,NaN,NaN,NaN
3,3,제39기 2006년 12월 31일 현재,NaN,NaN,NaN
4,4,제38기 2005년 12월 31일 현재,NaN,NaN,NaN
5,5,(단위 : 백만원),NaN,NaN,NaN
6,6,NaN,NaN,NaN,NaN
7,7,과 목,제40기 분기말,제39기말,제38기말
8,8,자산,,,
9,9,Ⅰ.유동자산,8428971,8043005,9003365


In [35]:
csv_temp = pd.read_csv(data_path + file, encoding ='cp949',index_col = 0) # index_col : index로 삼을 열 지정
csv_temp

,Unnamed: 0.1,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,연결 재무상태표,NaN,NaN,NaN
1,제 52 기 2019.12.31 현재,NaN,NaN,NaN
2,제 51 기 2018.12.31 현재,NaN,NaN,NaN
3,제 50 기 2017.12.31 현재,NaN,NaN,NaN
4,(단위 : 백만원),NaN,NaN,NaN
...,...,...,...,...
62,이익잉여금,68249633,66490082,67332328
63,매각예정자산과 관련된 자본,,1122,
64,비지배지분,6299952,5922041,5653870
65,자본총계,76365754,73896010,74757354


- step 1 단위가 적혀 있는 cell 찾기
- step 2 그 위로는 제거하고 단위에서 숫자로 추출하여 unit column에 붙이기
- step 3 unit을 곱하여 실제 숫자가 들어간 테이블 만들기
- step 3-1. 값의 항목명은 text만 들어간 형태로 모두 수정
- step 4 테이블에서 원하는 항목을 추출하여 filtered_raw 폴더에 저장하기 (가능하면 ~기 라는 단어 대신 년도 기입할 것)

In [41]:
for i in range(len(csv_temp.index)):
    if '단위' in csv_temp.iat[i,0]:
        slice_point = i
        break

In [149]:
# 원 단위로 하면 보기가 불편하여 억 단위로 반영
unit_korean = csv_temp.iat[i,0]
if '십억원' in unit_korean:
    unit = 10**9 / 10**8
elif'백만원' in unit_korean: 
    unit = 10**6 / 10**8
elif '천원' in unit_korean:
    unit = 10**3 / 10**8
else:
    print('check unit_korean')

In [150]:
csv_temp_filtered = csv_temp.iloc[i+1:,:]
csv_temp_filtered.columns = csv_temp_filtered.iloc[0,:]
csv_temp_filtered.index = csv_temp_filtered.iloc[:,0]
csv_temp_filtered = csv_temp_filtered.iloc[1:,1:]
# csv_temp_filtered = csv_temp_filtered.reset_index(drop=True)
csv_temp_filtered.columns.name=''

In [154]:
#빈칸에 있는 값은 결측치가 아닌 ' '이다.
csv_temp_filtered = csv_temp_filtered.replace(' ',0)
csv_temp_filtered = csv_temp_filtered.astype(float)
csv_temp_filtered = csv_temp_filtered * unit

In [155]:
csv_temp_filtered

,제 52 기,제 51 기,제 50 기
,,,
자산,0.00,0.00,0.00
유동자산,760828.73,730081.01,739759.14
현금및현금성자산,86819.71,91136.25,88215.29
단기금융상품,72926.26,79363.19,77458.29
기타금융자산,94499.13,97557.25,128867.69
...,...,...,...
이익잉여금,682496.33,664900.82,673323.28
매각예정자산과 관련된 자본,0.00,11.22,0.00
비지배지분,62999.52,59220.41,56538.70


# ETC

In [ ]:
# i=4
for i in range(10):
    while True:
        if i != 6:
            if i == 8:
                print('break')
                break
            i=i+1
        else:
            print(i)
            break